In [ ]:
%pip install -Uq openai-agents

In [ ]:
# Make your Jupyter Notebook capable of running asynchronous functions.

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig

In [ ]:
from google.colab import  userdata

gemini_api_key = userdata.get("GOOGLE_API_KEY")

# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)


config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

#agent with sync runner

In [ ]:
agent : Agent = Agent(name="maths-tutor", instructions = "you are a maths teacher for high school",model = model)
result = Runner.run_sync(agent , "solve 2+2 equation in statistics", run_config=config)
print(result.final_output)


Alright, let's talk about how "2 + 2" might show up in statistics, even though it's not typically a statistical equation in itself. The beauty of statistics is that simple arithmetic can underpin more complex concepts.

**Understanding the Context is Key**

The first thing to realize is that "2 + 2 = 4" is a fundamental arithmetic truth.  In statistics, it's *how* we apply this basic arithmetic that makes it relevant.  Here are some scenarios where "2 + 2" could indirectly be used in a statistical context:

**Scenario 1: Simple Frequency Counts**

*   **Problem:** You're collecting data on the number of students who prefer different types of fruit. 2 students like apples, and 2 students like bananas. How many students like apples or bananas?
*   **Solution:** 2 (apples) + 2 (bananas) = 4 students

**Scenario 2:  Calculating a Total**

*   **Problem:** You're calculating the total number of successes in a small experiment. In the first trial, you had 2 successes. In the second trial, yo

# agent with async runner

In [ ]:
import asyncio
from agents import Agent , Runner
from agents.run import RunConfig


async def main():
  agent = Agent(name="maths-tutor", instructions = "you are a maths teacher for high school")
  result = await Runner.run(agent , "solve 2+2 equation in statistics", run_config=config)
  print(result.final_output)

if __name__=="__main__":
  asyncio.run(main())

Okay, let's clarify something first.  The equation "2+2" by itself isn't really a statistical problem.  Statistics deals with collecting, analyzing, interpreting, and presenting data.  So, the *equation* 2+2 doesn't really fit into that context.

However, I think you might be hinting at a few possible scenarios where something that *looks* like "2+2" *could* come up within a statistical problem. Let's explore some possibilities:

**Scenario 1:  Simple Frequency/Counting**

*   **Context:**  You're collecting data on something.  For example, you're surveying students about their favorite color.  2 students say "Blue", and 2 students say "Green".
*   **Equation:** 2 (Blue) + 2 (Green) = 4 (Total number of students who chose blue or green)
*   **Statistical Relevance:** This is very basic data summarization. You're just counting occurrences and adding them up. You could then calculate percentages (e.g., 50% chose blue or green if there were 8 students total).

**Scenario 2:  Averaging/Mea

#Agent with Streeming

In [ ]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GOOGLE_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [ ]:
import asyncio

from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="researcher",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 top technologies of 2025 .", run_config=config)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)



asyncio.run(main())

Okay, predicting the future is always tricky, but based on current trends and expert forecasts, here are 5 technologies that are likely to be prominent and impactful in 2025:

1.  **Artificial Intelligence (AI) and Machine Learning (ML):**
    *   **Why:** AI and ML will be even more deeply integrated into nearly every aspect of life and work. Expect advancements in areas like:
        *   **Natural Language Processing (NLP):** More sophisticated chatbots, virtual assistants, and automated content creation.
        *   **Computer Vision:** Improved object recognition, facial recognition, and autonomous systems (e.g., self-driving cars).
        *   **AI-driven Automation:** Greater automation of tasks in industries like manufacturing, healthcare, and customer service.
        *   **Generative AI:** Tools that can create realistic images, videos, music, and text will become more widespread.

2.  **5G and Advanced Connectivity:**
    *   **Why:** 5G will be more widely deployed and matur

In [ ]:
import asyncio
import random

from agents import Agent, ItemHelpers, Runner, function_tool


@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
        run_config=config

    )
    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types





asyncio.run(main())

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 10
-- Message output:
 I am programmed to tell 10 jokes.

=== Run complete ===


# Tool

In [34]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GOOGLE_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-1.5-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [35]:
from agents.tool import function_tool

In [36]:
@function_tool("Student_finder")
def Student_finder(student_roll: int) -> str:
  data={
      1: 'bisma',
      2:'noor',
      3:'shafe',
      4:'sumbal',
  }
  return data.get(student_roll,"student not found")


In [37]:
@function_tool("get_weather")
def get_weather(location: str, unit: str = "C") -> str:
  """
  Fetch the weather for a given location, returning a short description.
  """
  # Example logic
  return f"The weather in {location} is 22 degrees {unit}."

In [40]:
import asyncio

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        tools=[get_weather, Student_finder], # add tools here
        model=model
    )

    result = await Runner.run(agent, "whats the temperature of lahore.")
    print(result.final_output)



if __name__ == "__main__":
    asyncio.run(main())

Hot sun shines above,
Lahore's temperature unknown,
API needs a key.

